In [1]:
import pandas as pd
import sqlite3 as sql

In [2]:
musicbox_conn = sql.connect("../data/musicbox.sqlite")
musicbox_conn.text_factory = str

In [19]:
pd.read_sql("SELECT * FROM sqlite_master", musicbox_conn)

,type,name,tbl_name,rootpage,sql
0,table,Down,Down,3,"CREATE TABLE Down(\n uid INT,\n device TEXT,..."
1,table,Play,Play,4,"CREATE TABLE Play(\n uid INT,\n device TEXT,..."
2,table,song_info,song_info,5,"CREATE TABLE ""song_info"" (\n""song_id"" REAL,\n ..."
3,table,Play_new,Play_new,6,"CREATE TABLE Play_new(\n uid INT,\n device T..."
4,table,churn,churn,7,"CREATE TABLE ""churn"" (\n""uid"" INTEGER,\n ""dev..."
5,index,songid_songlength_idx,Play,8,CREATE INDEX songid_songlength_idx ON Play (so...
6,index,song_id_name_idx,song_info,9,CREATE INDEX song_id_name_idx ON song_info(son...
7,index,date_uid_idx,Play_new,10,"CREATE INDEX date_uid_idx ON Play_new (date, uid)"


In [4]:
pd.read_sql("SELECT * FROM Play_new LIMIT 5", musicbox_conn)

,uid,device,song_id,song_name,singer,play_time,song_length,date
0,0,ar,0.0,花雨,張清,40384.0,297.0,2017-04-08
1,0,ar,0.0,花雨,張清,4974.0,297.0,2017-04-08
2,0,ar,0.0,花雨,張清,40384.0,297.0,2017-04-08
3,0,ar,0.0,花雨,張清,4974.0,297.0,2017-04-08
4,0,ar,0.0,花雨,張清,40384.0,297.0,2017-04-08


In [6]:
pd.read_sql("SELECT COUNT(*) FROM Play_new", musicbox_conn)

,COUNT(*)
0,163589120


In [7]:
pd.read_sql("SELECT COUNT(*) FROM Play", musicbox_conn)

,COUNT(*)
0,164400821


## Extract information from play data

In [37]:
# extremely large play_time
# first aggregate date per day, but would be problematic when deal with average play percentage
"""
SELECT
    uid, device,
    SUM(CASE WHEN date BETWEEN '2017-03-30' AND '2017-04-09' THEN freq ELSE NULL END) AS w1_freq,
    SUM(CASE WHEN date BETWEEN '2017-03-30' AND '2017-04-09' THEN total_play_time ELSE NULL END) AS w1_play_time,

    SUM(CASE WHEN date BETWEEN '2017-04-10' AND '2017-04-20' THEN freq ELSE NULL END) AS w2_freq,
    SUM(CASE WHEN date BETWEEN '2017-04-10' AND '2017-04-20' THEN total_play_time ELSE NULL END) AS w2_play_time,
    
    SUM(CASE WHEN date BETWEEN '2017-04-21' AND '2017-05-01' THEN freq ELSE NULL END) AS w3_freq,
    SUM(CASE WHEN date BETWEEN '2017-04-21' AND '2017-05-01' THEN total_play_time ELSE NULL END) AS w3_play_time,
    
    SUM(CASE WHEN date BETWEEN '2017-05-02' AND '2017-05-12' THEN freq ELSE NULL END) AS w4_freq,
    SUM(CASE WHEN date BETWEEN '2017-05-02' AND '2017-05-12' THEN total_play_time ELSE NULL END) AS w4_play_time,
FROM
    (
    SELECT 
        uid, date, device
        COUNT(1) AS freq, 
        SUM(play_time) AS total_play_time,
        SUM(CASE WHEN play_time / song_length  > 1 THEN 1.0 ELSE play_time / song_length END) AS sum_daily_play_pct,
    FROM Play
    GROUP BY 1, 2 
    ) sub
GROUP BY
    1
"""

CPU times: user 4.87 s, sys: 226 ms, total: 5.09 s
Wall time: 5.12 s


In [8]:
%%time
musicbox_conn.execute("CREATE INDEX date_uid_idx ON Play_new (date, uid);")

CPU times: user 8min 46s, sys: 39 s, total: 9min 25s
Wall time: 10min 47s


In [34]:
%%time
churn_df = pd.read_sql("""
SELECT
    uid, device,
    SUM(CASE WHEN time_window = 'w1' THEN freq ELSE NULL END) AS freq_w1,
    SUM(CASE WHEN time_window = 'w1' THEN total_play_time ELSE NULL END) AS play_time_w1,
    SUM(CASE WHEN time_window = 'w1' THEN avg_play_pct ELSE NULL END) AS avg_play_pct_w1,
    
    SUM(CASE WHEN time_window = 'w2' THEN freq ELSE NULL END) AS freq_w2,
    SUM(CASE WHEN time_window = 'w2' THEN total_play_time ELSE NULL END) AS play_time_w2,
    SUM(CASE WHEN time_window = 'w2' THEN avg_play_pct ELSE NULL END) AS avg_play_pct_w2,
    
    SUM(CASE WHEN time_window = 'w3' THEN freq ELSE NULL END) AS freq_w3,
    SUM(CASE WHEN time_window = 'w3' THEN total_play_time ELSE NULL END) AS play_time_w3,
    SUM(CASE WHEN time_window = 'w3' THEN avg_play_pct ELSE NULL END) AS avg_play_pct_w3,
    
    SUM(CASE WHEN time_window = 'w4' THEN freq ELSE NULL END) AS freq_w4,
    SUM(CASE WHEN time_window = 'w4' THEN total_play_time ELSE NULL END) AS play_time_w4,
    SUM(CASE WHEN time_window = 'w4' THEN avg_play_pct ELSE NULL END) AS avg_play_pct_w4
    
FROM
    (
    SELECT 
        uid,
        CASE WHEN date BETWEEN '2017-03-30' AND '2017-04-09' THEN 'w1'
             WHEN date BETWEEN '2017-04-10' AND '2017-04-20' THEN 'w2'
             WHEN date BETWEEN '2017-04-21' AND '2017-05-01' THEN 'w3'
             WHEN date BETWEEN '2017-05-02' AND '2017-05-12' THEN 'w4' END AS time_window,
        device,
        COUNT(1) AS freq,
        SUM(play_time) AS total_play_time,
        AVG(CASE WHEN play_time / song_length  > 1 THEN 1.0 ELSE play_time / song_length END) AS avg_play_pct  
    FROM Play_new
    WHERE
        date > '2017-03-29'
    GROUP BY 1, 2
    ) sub

GROUP BY uid
""", musicbox_conn)

CPU times: user 14min 7s, sys: 8min 22s, total: 22min 29s
Wall time: 41min 43s


In [35]:
churn_df.head()

,uid,device,freq_w1,play_time_w1,avg_play_pct_w1,freq_w2,play_time_w2,avg_play_pct_w2,freq_w3,play_time_w3,avg_play_pct_w3,freq_w4,play_time_w4,avg_play_pct_w4
0,0,ar,145338.0,1.251728e+10,0.695687,406719.0,1.750280e+10,0.590655,315694.0,2.000722e+10,0.682264,258804.0,2.123130e+10,0.711662
1,12333,ip,45.0,9.027000e+03,0.949343,114.0,2.437200e+04,0.900190,109.0,2.479000e+04,0.972819,75.0,1.462400e+04,0.931983
2,36816,ar,4.0,1.846110e+05,1.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,51923,ar,NaN,NaN,NaN,2.0,1.020000e+02,0.325743,NaN,NaN,NaN,NaN,NaN,NaN
4,60183,ar,354.0,1.354700e+04,0.148254,283.0,1.478100e+04,0.202751,333.0,1.539700e+04,0.179196,38.0,4.949000e+03,0.508749


In [36]:
churn_df.to_sql('churn', musicbox_conn, if_exists="replace", index=False)

In [115]:
%%time
last_play_date = pd.read_sql("""
SELECT 
    uid, MAX(date) AS last_play_date
FROM Play_new
WHERE date BETWEEN '2017-03-30' AND '2017-05-01'
GROUP BY uid
""", musicbox_conn)

CPU times: user 2min 47s, sys: 8.98 s, total: 2min 56s
Wall time: 3min 1s


In [45]:
churn_play = pd.read_sql("SELECT * FROM churn", musicbox_conn)

In [46]:
churn_play.shape

(594252, 14)

In [47]:
churn_play.head()

,uid,device,freq_w1,play_time_w1,avg_play_pct_w1,freq_w2,play_time_w2,avg_play_pct_w2,freq_w3,play_time_w3,avg_play_pct_w3,freq_w4,play_time_w4,avg_play_pct_w4
0,0,ar,145338.0,1.251728e+10,0.695687,406719.0,1.750280e+10,0.590655,315694.0,2.000722e+10,0.682264,258804.0,2.123130e+10,0.711662
1,12333,ip,45.0,9.027000e+03,0.949343,114.0,2.437200e+04,0.900190,109.0,2.479000e+04,0.972819,75.0,1.462400e+04,0.931983
2,36816,ar,4.0,1.846110e+05,1.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,51923,ar,NaN,NaN,NaN,2.0,1.020000e+02,0.325743,NaN,NaN,NaN,NaN,NaN,NaN
4,60183,ar,354.0,1.354700e+04,0.148254,283.0,1.478100e+04,0.202751,333.0,1.539700e+04,0.179196,38.0,4.949000e+03,0.508749


In [48]:
churn_play = churn_play[~(churn_play.freq_w1.isnull() & churn_play.freq_w2.isnull() & churn_play.freq_w3.isnull())]

In [49]:
churn_play.shape

(583962, 14)

In [116]:
last_play_date.shape

(583962, 2)

## Extract information from download data

In [8]:
pd.read_sql("SELECT * FROM Down LIMIT 5", musicbox_conn)

,uid,device,song_id,song_name,singer,date
0,168019810,ar,442554.0,小酒窝,林俊杰&蔡卓妍,2017-03-30
1,168019810,ar,6334611.0,社会摇,萧全,2017-03-30
2,168019810,ar,9867382.0,台阶,武艺,2017-03-30
3,168019810,ar,6660691.0,一次就好-(电影《夏洛特烦恼》暖水曲),杨宗纬,2017-03-30
4,168019810,ar,157606.0,一路上有你,张学友,2017-03-30


In [10]:
pd.read_sql("SELECT COUNT(DISTINCT uid) FROM Down", musicbox_conn)

,COUNT(DISTINCT uid)
0,252835


In [20]:
%%time
musicbox_conn.execute("CREATE INDEX down_date_uid_idx ON Down(date, uid);")

CPU times: user 19.2 s, sys: 1.32 s, total: 20.5 s
Wall time: 21.6 s


In [22]:
%%time
churn_down = pd.read_sql("""
SELECT
    uid,
    SUM(CASE WHEN time_window = 'w1' THEN down_freq ELSE NULL END) AS down_freq_w1,
    SUM(CASE WHEN time_window = 'w1' THEN down_uniq_song_num ELSE NULL END) AS down_uniq_song_num_w1,
    SUM(CASE WHEN time_window = 'w1' THEN down_uniq_singer_num ELSE NULL END) AS down_uniq_singer_num_w1,

    SUM(CASE WHEN time_window = 'w2' THEN down_freq ELSE NULL END) AS down_freq_w2,
    SUM(CASE WHEN time_window = 'w2' THEN down_uniq_song_num ELSE NULL END) AS down_uniq_song_num_w2,
    SUM(CASE WHEN time_window = 'w2' THEN down_uniq_singer_num ELSE NULL END) AS down_uniq_singer_num_w2,
    
    SUM(CASE WHEN time_window = 'w3' THEN down_freq ELSE NULL END) AS down_freq_w3,
    SUM(CASE WHEN time_window = 'w3' THEN down_uniq_song_num ELSE NULL END) AS down_uniq_song_num_w3,
    SUM(CASE WHEN time_window = 'w3' THEN down_uniq_singer_num ELSE NULL END) AS down_uniq_singer_num_w3,
    
    SUM(CASE WHEN time_window = 'w4' THEN down_freq ELSE NULL END) AS down_freq_w4,
    SUM(CASE WHEN time_window = 'w4' THEN down_uniq_song_num ELSE NULL END) AS down_uniq_song_num_w4,
    SUM(CASE WHEN time_window = 'w4' THEN down_uniq_singer_num ELSE NULL END) AS down_uniq_singer_num_w4
    
FROM
    (
    SELECT 
        uid,
        CASE WHEN date BETWEEN '2017-03-30' AND '2017-04-09' THEN 'w1'
             WHEN date BETWEEN '2017-04-10' AND '2017-04-20' THEN 'w2'
             WHEN date BETWEEN '2017-04-21' AND '2017-05-01' THEN 'w3'
             WHEN date BETWEEN '2017-05-02' AND '2017-05-12' THEN 'w4' END AS time_window,
        COUNT(1) AS down_freq,
        COUNT(DISTINCT song_id) AS down_uniq_song_num,
        COUNT(DISTINCT singer) AS down_uniq_singer_num
    FROM Down
    WHERE
        date > '2017-03-29'
    GROUP BY 1, 2
    ) sub

GROUP BY uid
""", musicbox_conn)

CPU times: user 47.7 s, sys: 3.62 s, total: 51.4 s
Wall time: 52.2 s


In [23]:
churn_down.shape

(252835, 13)

In [24]:
churn_down.head()

,uid,down_freq_w1,down_uniq_song_num_w1,down_uniq_singer_num_w1,down_freq_w2,down_uniq_song_num_w2,down_uniq_singer_num_w2,down_freq_w3,down_uniq_song_num_w3,down_uniq_singer_num_w3,down_freq_w4,down_uniq_song_num_w4,down_uniq_singer_num_w4
0,0,19926.0,5088.0,1700.0,60529.0,15685.0,3632.0,31647.0,9056.0,2279.0,28038.0,7275.0,1925.0
1,36816,2.0,2.0,2.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,104057,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,13.0,13.0,11.0
3,126721,8.0,8.0,3.0,3.0,3.0,3.0,NaN,NaN,NaN,1.0,1.0,1.0
4,250691,4.0,4.0,4.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [34]:
%%time
last_down_date = pd.read_sql("""
SELECT 
    uid, MAX(date) AS last_down_date
FROM Down
WHERE date < '2017-05-02'
GROUP BY uid
""", musicbox_conn)

CPU times: user 8.82 s, sys: 379 ms, total: 9.2 s
Wall time: 9.25 s


In [35]:
last_down_date.head()

,uid,last_down_date
0,0,2017-05-01
1,36816,2017-03-30
2,126721,2017-04-14
3,250691,2017-04-04
4,269510,2017-04-24


In [36]:
last_down_date.shape

(243009, 2)

In [38]:
churn_down = \
churn_down[~(churn_down.down_freq_w1.isnull() & churn_down.down_freq_w2.isnull() & churn_down.down_freq_w3.isnull())]

In [39]:
churn_down.shape
# of uid matches after eliminating all those inactive users in the first three windows

(243009, 13)

## Remove bot records and merge all the information

In [105]:
last_down_date['down_recency'] = pd.to_datetime('2017-05-02') - pd.to_datetime(last_down_date.last_down_date)

In [107]:
last_down_date.down_recency = last_down_date.down_recency.dt.days

In [111]:
last_down_date.drop(['last_down_date'], axis=1, inplace=True)

In [120]:
last_play_date['play_recency'] = pd.to_datetime('2017-05-02') - pd.to_datetime(last_play_date.last_play_date)

In [121]:
last_play_date.play_recency = last_play_date.play_recency.dt.days

In [124]:
last_play_date.drop(['last_play_date'], axis=1, inplace=True)

In [131]:
churn_play = churn_play.merge(last_play_date, on='uid')

In [134]:
churn_down = churn_down.merge(last_down_date, on='uid')

In [145]:
churn_play.fillna(0, inplace=True)
churn_down.fillna(0, inplace=True)

### pinpoint the bot...

In [173]:
# average play time... There are lots of outliers...Seems to be very unreliable information
pd.read_sql("SELECT AVG(play_time) FROM Play_new", musicbox_conn)

,AVG(play_time)
0,206543.034391


In [175]:
# average song_length
pd.read_sql("SELECT AVG(song_length) FROM song_info", musicbox_conn)

,AVG(song_length)
0,390.617008


In [176]:
print "Thus a person can only play {} songs in a window at most on average".format(11*24*3600/390.617008)

Thus a person can only play 2433.07377952 songs in a window at most on average


In [183]:
churn_play[churn_play.columns[churn_play.columns.str.contains("freq")]].quantile(0.999)
# the 0.999 quantile seems to be a fair but a little conservative cutoff

freq_w1    1803.039
freq_w2    1583.000
freq_w3    1505.039
freq_w4    1462.117
Name: 0.999, dtype: float64

In [186]:
churn_play.drop(['device'], axis=1).quantile(0.999)

uid                1.691270e+08
freq_w1            1.803039e+03
play_time_w1       2.469632e+07
avg_play_pct_w1    1.000000e+00
freq_w2            1.583000e+03
play_time_w2       1.809318e+07
avg_play_pct_w2    1.000000e+00
freq_w3            1.505039e+03
play_time_w3       1.262647e+07
avg_play_pct_w3    1.000000e+00
freq_w4            1.462117e+03
play_time_w4       8.559633e+06
avg_play_pct_w4    1.000000e+00
play_recency       3.300000e+01
Name: 0.999, dtype: float64

In [198]:
churn_play_clean \
    = churn_play[(churn_play.freq_w1 < churn_play.freq_w1.quantile(0.999)) & \
                 (churn_play.freq_w2 < churn_play.freq_w2.quantile(0.999)) & \
                 (churn_play.freq_w3 < churn_play.freq_w3.quantile(0.999)) & \
                 (churn_play.freq_w4 < churn_play.freq_w4.quantile(0.999)) & \
                 (churn_play.play_time_w1 < churn_play.freq_w1.quantile(0.999) * 390) & \
                 (churn_play.play_time_w2 < churn_play.freq_w2.quantile(0.999) * 390) & \
                 (churn_play.play_time_w3 < churn_play.freq_w3.quantile(0.999) * 390) & \
                 (churn_play.play_time_w4 < churn_play.freq_w4.quantile(0.999) * 390)]

In [221]:
churn_play_clean.describe()

,uid,freq_w1,play_time_w1,avg_play_pct_w1,freq_w2,play_time_w2,avg_play_pct_w2,freq_w3,play_time_w3,avg_play_pct_w3,freq_w4,play_time_w4,avg_play_pct_w4,play_recency
count,5.721710e+05,572171.000000,572171.000000,572171.000000,572171.000000,572171.000000,572171.000000,572171.000000,572171.000000,572171.000000,572171.000000,572171.000000,572171.000000,572171.000000
mean,1.674299e+08,69.199802,10416.298428,0.503286,42.292421,6721.031741,0.271756,37.188515,5866.262451,0.235344,32.324550,5131.638262,0.198535,17.790486
std,1.013930e+07,137.286739,28593.369710,0.304086,111.595663,21297.449539,0.340976,106.179785,19763.713840,0.330501,101.013392,18419.505639,0.315848,12.740300
min,1.233300e+04,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000
25%,1.680338e+08,4.000000,288.000000,0.263027,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,4.000000
50%,1.684903e+08,19.000000,2103.000000,0.527929,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,18.000000
75%,1.687731e+08,72.000000,9069.000000,0.755324,30.000000,3616.500000,0.580811,20.000000,2244.000000,0.513818,10.000000,1114.500000,0.408638,32.000000
max,1.692623e+08,1803.000000,702812.000000,1.000000,1582.000000,617032.000000,1.000000,1505.000000,586629.000000,1.000000,1462.000000,565866.000000,1.000000,33.000000


In [202]:
churn_play_clean.drop(['device'], axis=1).quantile(0.999)
# the values have become much more reasonable

uid                1.691276e+08
freq_w1            1.338830e+03
play_time_w1       3.824063e+05
avg_play_pct_w1    1.000000e+00
freq_w2            1.147830e+03
play_time_w2       2.493060e+05
avg_play_pct_w2    1.000000e+00
freq_w3            1.109830e+03
play_time_w3       2.313979e+05
avg_play_pct_w3    1.000000e+00
freq_w4            1.091000e+03
play_time_w4       2.176309e+05
avg_play_pct_w4    1.000000e+00
play_recency       3.300000e+01
Name: 0.999, dtype: float64

In [209]:
print "{:0.2f}% of play data are reserved.".format(100.0 * churn_play_clean.shape[0] / churn_play.shape[0])

97.98% of play data are reserved.


In [141]:
churn_down.sort_values('down_freq_w1', ascending=False).head()

,uid,down_freq_w1,down_uniq_song_num_w1,down_uniq_singer_num_w1,down_freq_w2,down_uniq_song_num_w2,down_uniq_singer_num_w2,down_freq_w3,down_uniq_song_num_w3,down_uniq_singer_num_w3,down_freq_w4,down_uniq_song_num_w4,down_uniq_singer_num_w4,down_recency
32,1685126,100701.0,24212.0,5410.0,66386.0,14743.0,3588.0,70089.0,16751.0,4347.0,47946.0,11439.0,3442.0,1
663,37025504,63027.0,15494.0,4071.0,46918.0,11604.0,3609.0,67872.0,16227.0,4001.0,43341.0,9830.0,3058.0,1
21,1062806,48090.0,14355.0,4263.0,32297.0,11054.0,3519.0,40016.0,12366.0,4281.0,25704.0,7905.0,3009.0,1
35,1791497,40578.0,10754.0,3217.0,33164.0,8849.0,2932.0,33994.0,8806.0,3103.0,35818.0,8862.0,3043.0,1
6,497685,34622.0,11528.0,3434.0,26648.0,9750.0,2752.0,27618.0,9485.0,2758.0,20934.0,7129.0,2413.0,1


In [222]:
churn_down_clean = churn_down[(churn_down.down_freq_w1 < churn_down.down_freq_w1.quantile(0.999)) & \
                              (churn_down.down_freq_w2 < churn_down.down_freq_w2.quantile(0.999)) & \
                              (churn_down.down_freq_w3 < churn_down.down_freq_w3.quantile(0.999)) & \
                              (churn_down.down_freq_w4 < churn_down.down_freq_w4.quantile(0.999))]

In [224]:
print "{:0.2f}% of download data are reserved.".format(100.0 * churn_down_clean.shape[0] / churn_down.shape[0])

99.67% of download data are reserved.


In [225]:
churn_down_clean.describe()

,uid,down_freq_w1,down_uniq_song_num_w1,down_uniq_singer_num_w1,down_freq_w2,down_uniq_song_num_w2,down_uniq_singer_num_w2,down_freq_w3,down_uniq_song_num_w3,down_uniq_singer_num_w3,down_freq_w4,down_uniq_song_num_w4,down_uniq_singer_num_w4,down_recency
count,2.422120e+05,242212.000000,242212.000000,242212.000000,242212.000000,242212.000000,242212.000000,242212.000000,242212.000000,242212.000000,242212.000000,242212.000000,242212.000000,242212.000000
mean,1.673911e+08,13.890658,13.385274,6.199350,3.742663,3.639386,1.815269,2.816491,2.733217,1.406487,1.851882,1.660269,0.874845,20.641141
std,1.058378e+07,35.976224,34.049147,14.795603,15.490904,14.932480,7.052829,12.632788,12.129416,5.990273,9.620203,8.578373,4.299022,11.494548
min,3.681600e+04,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000
25%,1.680341e+08,1.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,10.000000
50%,1.684778e+08,3.000000,3.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,24.000000
75%,1.687976e+08,12.000000,12.000000,6.000000,1.000000,1.000000,1.000000,1.000000,1.000000,0.000000,0.000000,0.000000,0.000000,32.000000
max,1.692623e+08,711.000000,701.000000,472.000000,354.000000,354.000000,245.000000,293.000000,293.000000,216.000000,259.000000,259.000000,206.000000,33.000000


In [226]:
churn_down_clean.uid.isin(churn_play_clean.uid).mean()
# left join or outer join ???

0.94249252720757026

In [228]:
print churn_play_clean.shape, churn_down_clean.shape

(572171, 15) (242212, 14)


### Join play and download data

In [242]:
churn_df = churn_play_clean.merge(churn_down_clean, how='left', on='uid')

In [243]:
print churn_df.shape, churn_play_clean.shape

(572171, 28) (572171, 15)


In [247]:
churn_df.down_recency = churn_df.down_recency.fillna(-999)

In [306]:
churn_df.down_recency = churn_df.down_recency.astype(int)

In [249]:
churn_df.fillna(0, inplace=True)

In [250]:
churn_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 572171 entries, 0 to 572170
Data columns (total 28 columns):
uid                        572171 non-null int64
device                     572171 non-null object
freq_w1                    572171 non-null float64
play_time_w1               572171 non-null float64
avg_play_pct_w1            572171 non-null float64
freq_w2                    572171 non-null float64
play_time_w2               572171 non-null float64
avg_play_pct_w2            572171 non-null float64
freq_w3                    572171 non-null float64
play_time_w3               572171 non-null float64
avg_play_pct_w3            572171 non-null float64
freq_w4                    572171 non-null float64
play_time_w4               572171 non-null float64
avg_play_pct_w4            572171 non-null float64
play_recency               572171 non-null int64
down_freq_w1               572171 non-null float64
down_uniq_song_num_w1      572171 non-null float64
down_uniq_singer_num_w1    

In [257]:
churn_df.device.value_counts()

ar    498866
ip     73294
mc         7
wp         4
Name: device, dtype: int64

In [283]:
churn_df['device_is_android'] = (churn_df.device == 'ar').astype(int)

In [292]:
churn_df['churn_label'] = ((churn_df.freq_w4 == 0) & (churn_df.down_freq_w4 == 0)).astype(int)

In [293]:
relevant_col = churn_df.columns[churn_df.columns.str.contains("android|1|2|3|recency|churn_label")]

In [294]:
print relevant_col

Index([u'freq_w1', u'play_time_w1', u'avg_play_pct_w1', u'freq_w2',
       u'play_time_w2', u'avg_play_pct_w2', u'freq_w3', u'play_time_w3',
       u'avg_play_pct_w3', u'play_recency', u'down_freq_w1',
       u'down_uniq_song_num_w1', u'down_uniq_singer_num_w1', u'down_freq_w2',
       u'down_uniq_song_num_w2', u'down_uniq_singer_num_w2', u'down_freq_w3',
       u'down_uniq_song_num_w3', u'down_uniq_singer_num_w3', u'down_recency',
       u'device_is_android', u'churn_label'],
      dtype='object')


In [295]:
churn_df.churn_label.mean()

0.6626096044713905

In [309]:
X = churn_df[relevant_col]

In [310]:
X.shape

(572171, 22)

In [311]:
X.head()

,freq_w1,play_time_w1,avg_play_pct_w1,freq_w2,play_time_w2,avg_play_pct_w2,freq_w3,play_time_w3,avg_play_pct_w3,play_recency,...,down_uniq_singer_num_w1,down_freq_w2,down_uniq_song_num_w2,down_uniq_singer_num_w2,down_freq_w3,down_uniq_song_num_w3,down_uniq_singer_num_w3,down_recency,device_is_android,churn_label
0,45.0,9027.0,0.949343,114.0,24372.0,0.900190,109.0,24790.0,0.972819,1,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-999,0,0
1,4.0,184611.0,1.000000,0.0,0.0,0.000000,0.0,0.0,0.000000,33,...,2.0,0.0,0.0,0.0,0.0,0.0,0.0,33,1,1
2,0.0,0.0,0.000000,2.0,102.0,0.325743,0.0,0.0,0.000000,18,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-999,1,1
3,354.0,13547.0,0.148254,283.0,14781.0,0.202751,333.0,15397.0,0.179196,1,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-999,1,0
4,13.0,780.0,0.239631,0.0,0.0,0.000000,10.0,1665.0,0.616586,9,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-999,1,1


In [312]:
X.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 572171 entries, 0 to 572170
Data columns (total 22 columns):
freq_w1                    572171 non-null float64
play_time_w1               572171 non-null float64
avg_play_pct_w1            572171 non-null float64
freq_w2                    572171 non-null float64
play_time_w2               572171 non-null float64
avg_play_pct_w2            572171 non-null float64
freq_w3                    572171 non-null float64
play_time_w3               572171 non-null float64
avg_play_pct_w3            572171 non-null float64
play_recency               572171 non-null int64
down_freq_w1               572171 non-null float64
down_uniq_song_num_w1      572171 non-null float64
down_uniq_singer_num_w1    572171 non-null float64
down_freq_w2               572171 non-null float64
down_uniq_song_num_w2      572171 non-null float64
down_uniq_singer_num_w2    572171 non-null float64
down_freq_w3               572171 non-null float64
down_uniq_song_num_w3   

In [313]:
X.to_csv('../data/churn_train.csv', index=False)